## Investigasi sampel data titanic berikut dengan cara :
1. Cek secara head, tail, sample, info lalu observasi apa yang bisa anda peroleh ?
2. Lakukan Statistical Summary dengan mengekstrak informasi yang didapat dari observasi anda ?
3. Cek apakah ada duplikat dan bagaimana handlenya ?
4. Cek apakah ada missing value, berapa persentasenya jika ada, dan bagaimana cara handlenya ?

## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [2]:
# import data
df = pd.read_excel('titanic.xlsx')
df.head()

,survived,name,sex,age
0,1,"Allen, Miss. Elisabeth Walton",female,29.0000
1,1,"Allison, Master. Hudson Trevor",male,0.9167
2,0,"Allison, Miss. Helen Loraine",female,2.0000
3,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000
4,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000


## EDA (Exploratory Data Analysis)

In EDA, i know we have to do five steps
1. Check Data Types
2. Check Missing Values
3. Check Duplicated
4. Check Inconsistent Data
5. Check Outlier

First, i need to know about the data. There are four columns (Survived, Name, Sex, and Age). Then, i have to know if the types is correct or there is something wrong that i can change.

### Handling Data Types

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   survived  500 non-null    int64  
 1   name      500 non-null    object 
 2   sex       500 non-null    object 
 3   age       451 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 15.8+ KB


Okay, I see the data types of each column. Then, i focus on sex, its actuallt only has two values (Female or Male)--Well, unless youre in Thailand, you can have many value in there, hehehe kidding. Okay, i think i can change its types to categorical (The reason? i want to train myself to think about data efficiency, i know this is a small dataset, but i want to get used to thinking in terms of big data. i can’t afford to be arbitrary with data types, since that would be inefficient in the long run.).

In [4]:
df['sex'] = df['sex'].astype('category')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   survived  500 non-null    int64   
 1   name      500 non-null    object  
 2   sex       500 non-null    category
 3   age       451 non-null    float64 
dtypes: category(1), float64(1), int64(1), object(1)
memory usage: 12.5+ KB


### Handling Miss Value

Second, I have to see about the missing value, I just use isnull() for this part because it is more efficient in pandas.

In [5]:
df.isnull().sum()

,0
survived,0
name,0
sex,0
age,49


I get it... there are 49 missing values in age, but I can't just fill them with mean/median, because I don't know about the distribution of the age column. I need to know about that.

In [6]:
df.describe()

,survived,age
count,500.000000,451.000000
mean,0.540000,35.917775
std,0.498897,14.766454
min,0.000000,0.666700
25%,0.000000,24.000000
50%,1.000000,35.000000
75%,1.000000,47.000000
max,1.000000,80.000000


Ohh, the median value is 35 and the mean value is 35.9. Interesting, this makes me understand the distribution and makes it clear that the distribution is a normal distribution, we can use the mean() to fill in the missing values.

In [7]:
df['age'] = df['age'].fillna(df['age'].mean())

In [8]:
df.isnull().sum()

,0
survived,0
name,0
sex,0
age,0


### Handling Duplicated

Okay, this is 3 out of 5 steps to clean data. In this part, we will see how to handle duplicated data. But before we start, I want to introduce to you about how Dibimbing makes me understand better how to see duplicated data with a simple formula which is data_without_duplication/total_data. I just found out about that, oh my god hahaha.

In [9]:
(len(df.drop_duplicates()))/len(df)

0.998

In [10]:
duplicates = df[df.duplicated(keep=False)]

In [11]:
duplicates

,survived,name,sex,age
104,1,"Eustis, Miss. Elizabeth Mussey",female,54.0
349,1,"Eustis, Miss. Elizabeth Mussey",female,54.0


In [12]:
duplicates_count = duplicates.groupby(list(df.columns)).size().reset_index().rename(columns={0:'Duplikat'})

<ipython-input-12-ef8418360a18>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  duplicates_count = duplicates.groupby(list(df.columns)).size().reset_index().rename(columns={0:'Duplikat'})


In [13]:
duplicates_count

,survived,name,sex,age,Duplikat
0,1,"Eustis, Miss. Elizabeth Mussey",female,54.0,2
1,1,"Eustis, Miss. Elizabeth Mussey",male,54.0,0


In [14]:
df = df.drop_duplicates()

In [15]:
(len(df.drop_duplicates()))/len(df)

1.0

### Handling Inconsistent Data

Okay, this is 4 out of 5 steps in cleaning the data. First, I know that df['survived'] only has two values which are 1 or 0, in the describe we know that there is no inconsistent data because min, max, quartile, and counts_values are normal, so I will skip about survived. Then we have to catch about age, in the describe, I see that min is 0.666 (Not logical for age), median is 35.9 (Not logical either), etc. So we will focus on age.

In [16]:
df.describe()

,survived,age
count,499.000000,499.000000
mean,0.539078,35.881538
std,0.498971,14.013337
min,0.000000,0.666700
25%,0.000000,25.500000
50%,1.000000,35.917775
75%,1.000000,45.000000
max,1.000000,80.000000


In [17]:
df['age'].value_counts().sort_index()

,count
age,
0.666700,1
0.833300,1
0.916700,1
1.000000,3
2.000000,1
3.000000,1
4.000000,2
6.000000,2
7.000000,1


Here, I know that the data is inconsistent if the data is less than 1 and the data is not an integer. So I will filter the data to see more details.

In [18]:
df[(df['age'] < 1) | (df['age'] % 1 != 0)].sort_values('age')

,survived,name,sex,age
428,1,"Hamalainen, Master. Viljo",male,0.666700
360,1,"Caldwell, Master. Alden Gates",male,0.833300
1,1,"Allison, Master. Hudson Trevor",male,0.916700
222,0,"Ovies y Rodriguez, Mr. Servando",male,28.500000
173,0,"Keeping, Mr. Edwin",male,32.500000
59,1,"Cassebeer, Mrs. Henry Arthur Jr (Eleanor Genev...",female,35.917775
46,0,"Cairns, Mr. Alexander",male,35.917775
15,0,"Baumann, Mr. John D",male,35.917775
74,0,"Clifford, Mr. George Quincy",male,35.917775
80,0,"Crafton, Mr. John Bertram",male,35.917775


Okay, I got it. I will change the data where when the age is less than 1, I will change it to 1 and if the data is not an integer, I will round the data.

In [19]:
df.loc[df['age'] < 1, 'age'] = 1

In [20]:
df[df['age'] < 1]

,survived,name,sex,age


In [21]:
df.loc[df['age'] % 1 != 0, 'age'] = df['age'].round()

In [22]:
df[df['age']%1 != 0]

,survived,name,sex,age


In [24]:
df.describe()

,survived,age
count,499.000000,499.000000
mean,0.539078,35.889780
std,0.498971,14.011967
min,0.000000,1.000000
25%,0.000000,25.500000
50%,1.000000,36.000000
75%,1.000000,45.000000
max,1.000000,80.000000


### Outlier Check

In [30]:
"""Actually we can see in describe for this variabel, beacuse its categorical
and its just have two option (1 or 0)"""
print(df['survived'].value_counts())

survived
1    269
0    230
Name: count, dtype: int64


In [33]:
Q1 = df['age'].quantile(0.25)
Q3 = df['age'].quantile(0.75)
IQR = Q3 - Q1

batas_atas = Q3 + (1.5 * IQR)
batas_bawah = Q1 - (1.5 * IQR)

outliers = df[(df['age'] > batas_atas) | (df['age'] < batas_bawah)]

In [34]:
outliers

,survived,name,sex,age
14,1,"Barkworth, Mr. Algernon Henry Wilson",male,80.0
61,1,"Cavendish, Mrs. Tyrell William (Julia Florence...",female,76.0


80 and 76 years old are normal ages for Titanic passengers, I will not delete the data.


---


## END

That's how to clean data. Thank you :)